# 라이브러리 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from datetime import datetime
import xgboost as xgb 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from category_encoders import TargetEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb 
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression          
from sklearn.model_selection import cross_val_score

plt.rcParams['font.family'] = 'Malgun Gothic'
pd.options.display.max_rows = None
pd.options.display.max_columns = None

# 데이터 불러오기

In [3]:
df = pd.read_csv("/Users/Jiwon/Documents/GitHub/practical_project/csv/스마트팜_수정데이터.csv", encoding="cp949")

# 데이터 전처리

## 농장아이디 "20261"은 더블링으로 drop

In [4]:
df_drop = df[df["농장아이디"]==20261].index
df.drop(df_drop, inplace=True)

## 불필요한 컬럼 제거 (동일한 행, 동일한 컬럼, null값 컬럼)

In [5]:
df.drop("체세포", axis=1, inplace=True)
df.drop("수집건수", axis=1, inplace=True)
df.drop("측정일시", axis=1, inplace=True)
df.drop("축종코드", axis=1, inplace=True)
df.drop("제조사 아이디", axis=1, inplace=True)
df.drop("수집일자", axis=1, inplace=True)

## 데이터 타입 변환

In [6]:
# 착유시작, 종료일시 datetime으로 전처리

df["착유시작일시"] = pd.to_datetime(df["착유시작일시"], format="%Y-%m-%d %H:%M")
df["착유종료일시"] = pd.to_datetime(df["착유종료일시"], format="%Y-%m-%d %H:%M")

In [7]:
# 혈액흐름 전처리

df["혈액흐름"] = df["혈액흐름"].apply(lambda x : 0 if x == "N" else 1)

# 파생변수 생성

In [8]:
# 착유시간(분) 

df["착유시간"] = df["착유종료일시"] - df["착유시작일시"]
df["착유시간"]  = df["착유시간"].dt.total_seconds() / 60  # 분 단위로 변환

In [9]:
# 개체별 나이 (개체번호 앞 7자리는 생년월일)

df_birth = df['개체번호'].astype(str).str[:8]
df_birthdt = pd.to_datetime(df_birth, format='%Y%m%d', errors='coerce')

def calculate_age(birth_date, current_date):

    # 만 나이 계산 로직
    # 현재 연도 - 출생 연도 - 불리언(생일이 현재 날짜보다 뒤에 있으면 1 빼기)
    age = current_date.year - birth_date.year - ((current_date.month, current_date.day) < (birth_date.month, birth_date.day))
    return age

current_analysis_date = datetime(2021, 10, 1)
df['나이'] = df_birthdt.apply(lambda x: calculate_age(x, current_analysis_date))

In [10]:
# 개체별 착유일수

df_count2 = df.groupby("개체번호")["측정일"].nunique().reset_index()
df_count2.rename(columns={"측정일": "개체별 착유일수"}, inplace=True)
df = df.merge(df_count2, on="개체번호", how="left")

KeyError: 'Column not found: 측정일'

In [ ]:
# P/F 비율 (유단백/유지방 비율)

df["P/F_ratio"] = df["유단백"]/ df["유지방"]

In [ ]:
# 공기흐름 비율 (착유시간에 따른 공기흐름)

df["공기흐름_비율"] = df["공기흐름"] / df["착유시간"]

# 결측치 제거

In [4]:
# 유지방, 유단백 결측치 제거 -> 같이 존재하기 때문에 유지방으로 제거

null_fat = df["유지방"].isnull()
null_fat_index = null_fat[null_fat].index
df.drop(null_fat_index, inplace=True)

NameError: name 'df' is not defined